# Question

There is a file containing market data for bitcoin. Convert it to SQLite and analyze the file and find:
- The earliest and the latest price
- The maximum price for each year

In [1]:
import json
import sqlite3
import  datetime

In [2]:
# Load JSON file

file = open(file='data/bitcoin_price.json')
raw_data = json.load(file)
file.close()

raw_data[0]

{'symbol': 'btcusd',
 'time': 1364688000000,
 'open': 92.5,
 'close': 93.033,
 'high': 93.74999,
 'low': 91.0,
 'volume': 3083.07979075}

In [3]:
# Check data structure

print(f'Data is a {type(raw_data)} of {type(raw_data[0])}')

Data is a <class 'list'> of <class 'dict'>


In [4]:
# Convert to DataFrame

data = []
for row in raw_data:
    data.append([row['time'] // 1000, row['open'], row['close'], row['high'], row['low'], row['volume']])

Data should be prepared as  follows:
- 'symbol' is redundant
- 'time' is a date in ms but since it is unique so should be kept as INTEGER and used as primary key
- 'open', 'close', 'high', 'low' and 'volume' are REAL

In [5]:
# Export data to SQLite database

connection = sqlite3.connect('data/bitcoin.sql')
cursor = connection.cursor()

command = 'CREATE TABLE IF NOT EXISTS price (time INTEGER PRIMARY KEY, open REAL, close REAL, high REAL, low REAL, volume REAL);'
cursor.execute(command)

command = 'INSERT INTO price VALUES (?, ?, ?, ?, ?, ?);'
cursor.executemany(command, data)

connection.commit()
connection.close()

In [6]:
# Read data from database

connection = sqlite3.connect('data/bitcoin.sql')
cursor = connection.cursor()

command = """
        SELECT MIN(time), open 
        FROM price;
        """
result = cursor.execute(command).fetchone()
print(f'First open price: {result[1]} on {datetime.datetime.fromtimestamp(result[0])}')

command = """
        SELECT MAX(time), close 
        FROM price;
        """
result = cursor.execute(command).fetchone()
print(f'Last close price: {result[1]} on {datetime.datetime.fromtimestamp(result[0])}')

command = """
        SELECT MAX(close), STRFTIME('%Y', DATE(time, 'unixepoch')) as year
        FROM price
        GROUP BY year
        ORDER BY year;
        """
result = cursor.execute(command).fetchall()
print('Yearly Data'.center(20, '-'))
for row in result:
    print(f'{row[1]}: {str(row[0]).rjust(14)}')


connection.close()

First open price: 92.5 on 2013-03-30 20:00:00
Last close price: 17686.840768 on 2020-11-27 19:00:00
----Yearly Data-----
2013:         1138.5
2014:         919.24
2015:         466.01
2016:          981.7
2017:        19210.0
2018:        17169.0
2019:        12876.0
2020:        19131.0


In [7]:
data

[[1364688000, 92.5, 93.033, 93.74999, 91.0, 3083.07979075],
 [1364774400, 93.25, 103.999, 105.9, 92.49999, 5224.40131297],
 [1364860800, 104.0, 118.22935407, 118.3867, 99.0, 8376.52747811002],
 [1364947200, 117.95826131, 134.7, 146.88, 101.51088, 12996.2450721801],
 [1365033600, 134.71656, 132.899, 143.0, 119.0, 6981.66830544001],
 [1365120000, 130.10001, 142.3, 145.0, 128.25, 5234.88220325001],
 [1365206400, 142.8, 143.05, 144.0, 139.0, 2009.12862494],
 [1365292800, 143.5, 162.304, 164.75, 142.75, 3725.06764496],
 [1365379200, 163.9, 186.985, 194.57, 163.2, 10895.82741442],
 [1365465600, 186.8, 232.99, 249.97, 186.8, 6073.37314608],
 [1365552000, 228.669, 162.0, 290.0, 19.9, 16070.6427442701],
 [1365638400, 162.0, 100.0, 205.0, 1.06, 16914.6563150801],
 [1365724800, 109.0, 112.0, 157.0, 51.0, 38693.9553043196],
 [1365811200, 112.0, 91.1, 130.0, 81.12, 23866.6770456],
 [1365897600, 91.1, 90.171, 109.0, 20.0, 16437.2196645],
 [1365984000, 89.86, 83.302, 104.0, 71.497, 16393.12856398],
 